In [1]:
import os
import numpy as np
import signal_processing as sp
import matplotlib.pyplot as plt
import time
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sys import getsizeof

In [2]:
# Import datasets
dataset_directory = os.path.join(os.getcwd(), "DeepSig-Dataset-2018")

# Use these to load the entire dataset into memory
fft_dataset_train = np.load(os.path.join(dataset_directory, "fft_dataset_train.npy"))
fft_dataset_val = np.load(os.path.join(dataset_directory, "fft_dataset_val.npy"))
fft_dataset_test = np.load(os.path.join(dataset_directory, "fft_dataset_test.npy"))

labels_train = np.load(os.path.join(dataset_directory, "labels_train.npy"))
labels_val = np.load(os.path.join(dataset_directory, "labels_val.npy"))
labels_test = np.load(os.path.join(dataset_directory, "labels_test.npy"))

# # Use these to use a memory map to save RAM
# fft_dataset_train = np.load(os.path.join(dataset_directory, "fft_dataset_train.npy"), mmap_mode="r+")
# fft_dataset_val = np.load(os.path.join(dataset_directory, "fft_dataset_val.npy"), mmap_mode="r+")
# fft_dataset_test = np.load(os.path.join(dataset_directory, "fft_dataset_test.npy"), mmap_mode="r+")

# labels_train = np.load(os.path.join(dataset_directory, "labels_train.npy"), mmap_mode="r+")
# labels_val = np.load(os.path.join(dataset_directory, "labels_val.npy"), mmap_mode="r+")
# labels_test = np.load(os.path.join(dataset_directory, "labels_test.npy"), mmap_mode="r+")

In [12]:
# Create a model constructor
def model_constructor():
    inputs = keras.Input(shape=(1024, 1))
    x = layers.Conv1D(filters=16, kernel_size=3, activation='relu')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=32, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128)(x)
    x = layers.Dense(128)(x)
    outputs = layers.Dense(24, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Define the callbacks and save the best model to a new file
    callbacks = [keras.callbacks.ModelCheckpoint(filepath='models/cnn_test.keras', save_best_only=True, monitor='val_loss'), 
        keras.callbacks.EarlyStopping(monitor="val_loss", min_delta = 0.01, patience = 5, verbose = 1, restore_best_weights = True)]

    return model, callbacks

In [11]:
model, callbacks = model_constructor()

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1024, 1)]         0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 1022, 16)          64        
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 511, 16)           0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 509, 32)           1568      
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 254, 32)           0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 252, 64)           6208      
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 126, 64)           0   

In [ ]:
# Time the time it takes to train the model
start = time.time()

history = model.fit(fft_dataset_shuffled, labels_shuffled, epochs=30, batch_size = 256, validation_split = 0.3, callbacks=callbacks)

print("--- Model trained in %s seconds ---" % (time.time() - start))